In [1]:
import pandas as pd
import re
import json
from typing import List
from sssom import Mapping, MappingSet
from IPython.core.display import HTML
import yaml
from typing import List
from sssom import Mapping, MappingSet
from sssom.sssom_document import MappingSetDocument
from sssom.parsers import to_mapping_set_dataframe
from sssom.writers import write_table

head_count=0 # or 40

# Inputs
pato_labels_file="../ontology/reports/pato.tsv"
uberon_labels_file="../ontology/reports/uberon.tsv"
efo_labels_file="../ontology/reports/efo.tsv"
oba_labels_file="../ontology/reports/oba.tsv"
cl_labels_file="../ontology/reports/cl.tsv"
go_labels_file="../ontology/reports/go.tsv"
vt_labels_file="../ontology/reports/vt.tsv"
chebi_labels_file="../ontology/reports/chebi.tsv"

oba_efo_mapping_file="../mappings/oba-efo.sssom.tsv"
oba_vt_mapping_file="../mappings/oba-vt.sssom.tsv"
oba_efo_mapping_metadata_file="../mappings/oba-efo.sssom.yml"
oba_vt_mapping_metadata_file="../mappings/oba-vt.sssom.yml"
oba_efo_exclusion_file="../mappings/oba-efo-mapping-exclusions.sssom.tsv" # Contains all terms we decided not to map for one reason or another
oba_vt_exclusion_file="../mappings/oba-vt-mapping-exclusions.sssom.tsv" # Contains all terms we decided not to map for one reason or another

# Outputs
oba_vt_mapping_candidates_file="../mappings/oba-vt-unreviewed.sssom.tsv" #VT mappings identified by pipeline but not reviewed
oba_vt_mapping_missed_file="../mappings/oba-vt-missed.sssom.tsv" #VT mappings identified by looking at OBA IRIs (no need for review)
oba_vt_mapping_unmapped_file="../mappings/oba-vt-unmapped.sssom.tsv" #VT terms that have not been mapped so far (excluding reviewed and candidate mappings)
oba_vt_dosdp_candidates_file="../mappings/oba-vt-unreviewed.dosdp.tsv" # VT terms with candidate DOSDP mappings

oba_efo_mapping_candidates_file="../mappings/oba-efo-unreviewed.sssom.tsv" # see above vt analog
oba_efo_mapping_unmapped_file="../mappings/oba-efo-unmapped.sssom.tsv" # see above vt analog
oba_efo_dosdp_candidates_file="../mappings/oba-efo-unreviewed.dosdp.tsv" # see above vt analog

# Generic printing function; all printouts are passed through here
def echo(string):
    print(string) 
    #pass


# OBA Alignment Work

Make sure that you update the source data

```
sh run.sh make prepare_oba_alignment
```

We are doing three seperate things here:

1. Aligning EFO with OBA. This involves matching EFO classes to VT and OBA classes with SSSOM
2. Trying to patternise EFO classes by matching qualities and entities in its labels
3. Aligning VT with OBA by trying to patternise VT classes that have not been included in OBA so far


## Reading all data

The only input to this simple process is a table with labels and exact synonyms for all participating ontologies:

For matching potential DOSDP patterns:

* PATO (Qualities)
* UBERON (Entities)
* CL (Entities)
* GO (Entities)

For matching traits in general:

* VT
* EFO
* OBA

In [2]:
def save_tsv(df, filename):
    if "index" in df.columns:
        df = df.drop("index", axis=1, errors='ignore')
    df.to_csv(filename, sep="\t", index=False)


def mentions_token(value, list_of_strings):
    for l in list_of_strings:
        if l in value:
            #echo(f"value: {value}: {l}")
            return True
    return False


def read_labels(fn, prefix, labels_must_match=[]):
    labels = pd.read_csv(fn, sep="\t", low_memory=False)
    labels = labels.astype(str)
    labels.columns = ['id', 'predicate', 'value', 'type']
    labels = labels[['id', 'predicate', 'value']]
    labels['id']=labels['id'].str.replace('<http://purl.obolibrary.org/obo/','', regex=False)
    labels['id']=labels['id'].str.replace('<http://www.ebi.ac.uk/efo/','', regex=False)
    labels['id']=labels['id'].str.replace('>','', regex=False)
    labels['id']=labels['id'].str.replace('_',':', regex=False)
    labels['predicate']=labels['predicate'].str.replace('<http://www.w3.org/2000/01/rdf-schema#','rdfs:', regex=False)
    labels['predicate']=labels['predicate'].str.replace('<http://www.geneontology.org/formats/oboInOwl#','oboInOwl:', regex=False)
    labels['predicate']=labels['predicate'].str.replace('>','', regex=False)
    labels=labels[labels['id'].str.contains(prefix)]
    if labels_must_match:
        labels=labels[labels['value'].apply(lambda v: mentions_token(v, labels_must_match))]
        #labels=labels[labels['value'].str.contains('|'.join(labels_must_match))]
    return labels

In [3]:
efo_labels=read_labels(efo_labels_file, "EFO")
efo_labels

,id,predicate,value
0,EFO:0010138,rdfs:label,nitrite measurement
1,EFO:0020165,rdfs:label,AT-rich interactive domain-containing protein ...
2,EFO:0008214,rdfs:label,lymphotactin measurement
3,EFO:0005273,rdfs:label,sleep depth
4,EFO:0010773,rdfs:label,CD5 measurement
...,...,...,...
4966,EFO:0005117,oboInOwl:hasExactSynonym,"Regulated on Activation, Normal T cell Express..."
4967,EFO:0009272,oboInOwl:hasExactSynonym,VCA-IgG seropositivity
4968,EFO:0009272,oboInOwl:hasExactSynonym,VCA seropositivity
4969,EFO:0010381,oboInOwl:hasExactSynonym,PC 36:3


In [4]:
vt_labels=read_labels(vt_labels_file, "VT")
vt_labels

,id,predicate,value
0,VT:0003477,rdfs:label,nerve fiber response trait
1,VT:0003799,rdfs:label,macrophage migration trait
2,VT:0005208,rdfs:label,iris stroma morphology trait
3,VT:0010765,rdfs:label,head and neck integrity trait
4,VT:0010508,rdfs:label,neurocranium mass
...,...,...,...
5995,VT:0002295,oboInOwl:hasExactSynonym,pulmonary circulation
5996,VT:0000953,oboInOwl:hasExactSynonym,oligoglia morphology trait
5997,VT:0005463,oboInOwl:hasExactSynonym,CD4+ T cell physiology trait
5998,VT:0005463,oboInOwl:hasExactSynonym,CD4+ T cell function


#### Tokens to reduce search space

Label tokens created to reduce search space for relevant entities. There is no point to searching through 100000 CHEBI enties to match every label if only 1000 relevant chebi entities theoretically exist. For CHEBI for example, this reduces the search space by 2/3.

In [5]:
# VT tokens
vt_label_tokens = []
[vt_label_tokens.extend(l) for l in vt_labels['value'].str.split()]

# EFO tokens
efo_label_tokens = []
[efo_label_tokens.extend(l) for l in efo_labels['value'].str.split()]
efo_label_tokens.append('quality')

label_tokens = list(set(vt_label_tokens+efo_label_tokens))
label_tokens = [token for token in label_tokens if len(token)>3]
len(label_tokens)

5984

#### Loading tables from EQ identification

In [6]:
pato_labels=read_labels(pato_labels_file, "PATO", label_tokens)
pato_labels

,id,predicate,value
4,PATO:0001519,rdfs:label,sound quality
14,PATO:0002215,rdfs:label,falciform
21,PATO:0000142,rdfs:label,obsolete substance
31,PATO:0000764,rdfs:label,passive
41,PATO:0002303,rdfs:label,decreased object quality
...,...,...,...
22042,PATO:0002045,oboInOwl:hasExactSynonym,dendriform
22075,PATO:0001624,oboInOwl:hasExactSynonym,having decreased function
22076,PATO:0001624,oboInOwl:hasExactSynonym,partial functionality
22077,PATO:0001624,oboInOwl:hasExactSynonym,low functionality


In [7]:
chebi_labels=read_labels(chebi_labels_file, "CHEBI", label_tokens)
chebi_labels

,id,predicate,value
0,CHEBI:96359,rdfs:label,"3-(1,3-benzodioxol-5-yl)-1-[[(4S,5S)-2-[(2R)-1..."
2,CHEBI:91284,rdfs:label,alpha-D-Man-(1->2)-alpha-D-Man-(1->2)-alpha-D-...
4,CHEBI:37429,rdfs:label,aldonolactone phosphate
7,CHEBI:153383,rdfs:label,"N-[(3R,4R,5S,6R)-5-[(2S,3R,4R,5S,6R)-3-Acetami..."
8,CHEBI:184015,rdfs:label,8-allyloxypsoralen
...,...,...,...
257986,CHEBI:77795,oboInOwl:hasExactSynonym,"1-(2,4-difluorophenyl)-6-fluoro-7-[(3S)-3-meth..."
257988,CHEBI:189596,oboInOwl:hasExactSynonym,[(2R)-3-henicosanoyloxy-2-hydroxypropyl] 2-(tr...
257989,CHEBI:179100,oboInOwl:hasExactSynonym,"methyl 2-hydroxy-2-methyl-1,3,5,6,7,8-hexahydr..."
257990,CHEBI:46195,oboInOwl:hasExactSynonym,N-(4-hydroxyphenyl)acetamide


In [8]:
uberon_labels=read_labels(uberon_labels_file, "UBERON", label_tokens)
uberon_labels

,id,predicate,value
1,UBERON:0007232,rdfs:label,2 cell stage
2,UBERON:0007713,rdfs:label,fourth sacral spinal ganglion
3,UBERON:4300020,rdfs:label,anal fin basal cartilage
6,UBERON:0009657,rdfs:label,artery of lip
7,UBERON:0002370,rdfs:label,thymus
...,...,...,...
49335,UBERON:0008874,oboInOwl:hasExactSynonym,arbor alveolaris
49336,UBERON:0008874,oboInOwl:hasExactSynonym,primary pulmonary lobule
49338,UBERON:0008874,oboInOwl:hasExactSynonym,respiratory lobule
49340,UBERON:0001999,oboInOwl:hasExactSynonym,iliopsoas muscle


In [9]:
cl_labels=read_labels(cl_labels_file, "CL", label_tokens)
cl_labels

,id,predicate,value
6,CL:1000380,rdfs:label,type 1 vestibular sensory cell of epithelium o...
17,CL:0005021,rdfs:label,mesenchymal lymphangioblast
19,CL:0000397,rdfs:label,ganglion interneuron
20,CL:1000291,rdfs:label,myocyte of posterior internodal tract
21,CL:0000878,rdfs:label,central nervous system macrophage
...,...,...,...
46681,CL:0000935,oboInOwl:hasExactSynonym,"CD4-negative, CD8-negative, alpha-beta intraep..."
46682,CL:0000935,oboInOwl:hasExactSynonym,"CD4-negative, CD8-negative, alpha-beta intraep..."
46683,CL:0000935,oboInOwl:hasExactSynonym,"CD4-negative, CD8-negative, alpha-beta intraep..."
46786,CL:0002670,oboInOwl:hasExactSynonym,type I spiral ligament fibrocyte


In [10]:
go_labels=read_labels(go_labels_file, "GO", label_tokens)
go_labels

,id,predicate,value
0,GO:0071389,rdfs:label,cellular response to mineralocorticoid stimulus
1,GO:0007561,rdfs:label,imaginal disc eversion
2,GO:0051685,rdfs:label,maintenance of ER location
3,GO:0034275,rdfs:label,kynurenic acid metabolic process
4,GO:0060870,rdfs:label,cell wall disassembly involved in floral organ...
...,...,...,...
122608,GO:0015997,oboInOwl:hasExactSynonym,ubiquinone formation monooxygenase activity
122609,GO:0015997,oboInOwl:hasExactSynonym,ubiquinone anabolism monooxygenase activity
122610,GO:0015997,oboInOwl:hasExactSynonym,coenzyme Q biosynthesis monooxygenase activity
122611,GO:0015997,oboInOwl:hasExactSynonym,coenzyme Q biosynthetic process monooxygenase ...


In [11]:
oba_labels=read_labels(oba_labels_file, "OBA")
oba_labels

,id,predicate,value
0,OBA:0005640,rdfs:label,philtrum amount
10,OBA:VT0009921,rdfs:label,transitional stage T3 B cell morphology trait
15,OBA:VT0010453,rdfs:label,abdominal wall mass
22,OBA:VT0005208,rdfs:label,iris stroma morphology trait
24,OBA:VT0001148,rdfs:label,testes size trait
...,...,...,...
73403,OBA:0002099,oboInOwl:hasExactSynonym,amount of sensory perception of smell
73404,OBA:0003173,oboInOwl:hasExactSynonym,symmetry of ear
73406,OBA:0003002,oboInOwl:hasExactSynonym,2-D shape of phalanx of manus
73409,OBA:1000249,oboInOwl:hasExactSynonym,quality of basicranium


### Loading mappings and exclusion file

In [12]:
from sssom.parsers import parse_sssom_table
with open(oba_efo_mapping_metadata_file, 'r') as stream:
    oba_efo_meta=yaml.safe_load(stream)
with open(oba_vt_mapping_metadata_file, 'r') as stream:
    oba_vt_meta=yaml.safe_load(stream)

efo_mapping_exclusions=pd.read_csv(oba_efo_exclusion_file,sep="\t")
vt_mapping_exclusions=pd.read_csv(oba_vt_exclusion_file,sep="\t")
oba_efo_sssom = parse_sssom_table(oba_efo_mapping_file, prefix_map=oba_efo_meta['curie_map'], meta=oba_efo_meta)
oba_vt_sssom = parse_sssom_table(oba_vt_mapping_file, prefix_map=oba_vt_meta['curie_map'], meta=oba_vt_meta)

In [13]:
oba_vt_sssom.df

,subject_id,predicate_id,object_id,match_type
0,VT:0000000,skos:exactMatch,OBA:VT0000000,HumanCurated
1,VT:0000002,skos:exactMatch,OBA:VT0000002,HumanCurated
2,VT:0000003,skos:exactMatch,OBA:VT0000003,HumanCurated
3,VT:0000005,skos:exactMatch,OBA:VT0000005,HumanCurated
4,VT:0000006,skos:exactMatch,OBA:VT0000006,HumanCurated
...,...,...,...,...
2298,VT:3000004,skos:exactMatch,OBA:VT3000004,HumanCurated
2299,VT:4000007,skos:exactMatch,OBA:VT4000007,HumanCurated
2300,VT:4000009,skos:exactMatch,OBA:VT4000009,HumanCurated
2301,VT:4000011,skos:exactMatch,OBA:VT4000011,HumanCurated


In [14]:
oba_efo_sssom.df

,subject_id,subject_label,subject_category,predicate_id,object_id,object_label,object_category,match_type,author_id,subject_source,object_source,mapping_date,confidence,subject_match_field,object_match_field,match_string
0,EFO:0003923,bone density,measurement,skos:exactMatch,OBA:1000110,bone density,quality,HumanCurated,orcid:0000-0001-8314-2140,EFO,OBA,2022-06-08,1.0,rdfs:label,rdfs:label,bone density
1,EFO:0004301,blood viscosity,measurement,skos:exactMatch,OBA:VT3000004,blood viscosity trait,quality,HumanCurated,orcid:0000-0001-8314-2140,EFO,OBA,2021-09-10,1.0,,,
2,EFO:0004325,blood pressure,measurement,skos:exactMatch,OBA:VT0000183,blood pressure trait,quality,HumanCurated,orcid:0000-0001-8314-2140,EFO,OBA,2021-09-10,1.0,,,
3,EFO:0004326,heart rate,measurement,skos:exactMatch,OBA:1001087,heart rate,quality,HumanCurated,orcid:0000-0001-8314-2140,EFO,OBA,2022-06-08,1.0,rdfs:label,rdfs:label,heart rate
4,EFO:0004339,body height,measurement,skos:exactMatch,OBA:VT0001253,body height,quality,HumanCurated,orcid:0000-0001-8314-2140,EFO,OBA,2022-06-08,1.0,rdfs:label,rdfs:label,body height
5,EFO:0004342,waist circumference,measurement,skos:exactMatch,OBA:1001085,waist circumference,quality,HumanCurated,orcid:0000-0001-8314-2140,EFO,OBA,2022-06-08,1.0,rdfs:label,rdfs:label,waist circumference
6,EFO:0004589,upper arm circumference,measurement,skos:exactMatch,OBA:1000030,upper arm circumference,quality,HumanCurated,orcid:0000-0001-8314-2140,EFO,OBA,2022-06-08,1.0,oboInOwl:hasExactSynonym,rdfs:label,upper arm circumference
7,EFO:0004629,von willebrand factor quality,measurement,skos:exactMatch,OBA:1000199,von Willebrand factor quality,quality,HumanCurated,orcid:0000-0001-8314-2140,EFO,OBA,2022-06-08,1.0,rdfs:label,rdfs:label,von willebrand factor quality
8,EFO:0004742,renal system quality,measurement,skos:exactMatch,OBA:VT1000777,renal system quality,,HumanCurated,orcid:0000-0001-8314-2140,EFO,OBA,2022-06-08,1.0,rdfs:label,oboInOwl:hasExactSynonym,renal system quality
9,EFO:0004766,subcutaneous adipose tissue quality,measurement,skos:exactMatch,OBA:1000023,subcutaneous adipose tissue quality,,HumanCurated,orcid:0000-0001-8314-2140,EFO,OBA,2022-06-08,1.0,rdfs:label,rdfs:label,subcutaneous adipose tissue quality


## Library of functions used for matching below

You dont have to read the library for now, skip it. It contains all methods needed for the matching process

In [15]:
# These are words we consider noise for the sake of this alignment process. Its probably worth adding more
stopwords = ["of", "in"]
replacements = {
    'measurement': 'quality',
    'trait': 'quality',
}

# Function that describes what we consider a "whole word" match in terms of regex. Probably
# Can be simplified.
def whole_word_regex(stopwords):
    stopwords_regex = []
    for stopword in stopwords:
        stopword = re.escape(stopword)
        stopwords_regex.append(f"[ .;,:]{stopword}[ .;,:]")
        stopwords_regex.append(f"[ ]{stopword}$")
        stopwords_regex.append(f"^{stopword}[ .;,:]")
        stopwords_regex.append(f"^{stopword}$")
    return stopwords_regex

# The label is prepared: lower cased, trimmed, stop words removed.
# Note this should probably do stemming, lemmatisation and _proper_ stop word removal (and, or, of) as well
def prepare_label(value, stopwords=[], replacements=replacements):
    stopwords_regex = whole_word_regex(stopwords)                           
    if isinstance(value, str):
        for regex_value in stopwords_regex:
            value = re.sub(regex_value, "", value)
        for replace_this in replacements:
            value = value.replace(replace_this, replacements[replace_this])
        value = value.lower()
        value = value.strip()
        return value
    else: 
        return ""


# Determines if the "value_to_match" is contained in label _as a whole word_. So active would not be 
# matched in a label which says inactive.
def whole_word_match(label, value_to_match, min_match_size=3):
    if value_to_match in label:
        whole_word_regexes = whole_word_regex([value_to_match,])
        for regex_value in whole_word_regexes:
            if(re.search(regex_value, label)):
                #echo(f"Match found: {regex_value} in {label}")
                return True
    return False

# the value (probably an OBA, VT or EFO label) is matched against everything in the dataframe df. 
# strict_word_order=False would split the words of the label into a list, then sorting them, which allows
# matching words which are simply changed in order, like "cell count" vs "count of cell"
def get_matches(value, df, strict_word_order=True, exact_only=True, min_match_size=3):
    matches = []
    if not strict_word_order:
        # This sorts the words in the string before attempting to match
        value = " ".join(sorted(value.split(" ")))
    
    for index, row in df.iterrows():
        curie = row['id']
        predicate = row['predicate']
        label = prepare_label(row['value'])
        if len(label)>min_match_size:
            if not strict_word_order:
                label = " ".join(sorted(label.split(" ")))
            if label==value:
                matches.append({"match_string": label, "object_id": curie , "object_label": row['value'], 'object_match_field' : predicate, 'predicate_id': 'skos:exactMatch' })
            elif not exact_only and whole_word_match(value, label, min_match_size):
                matches.append({"match_string": label, "object_id": curie , "object_label": row['value'], 'object_match_field' : predicate, 'predicate_id': 'skos:relatedMatch' })
    
    return matches

def print_matches(matches):
    if matches:
        echo("Matches:")
        for q in matches:
            l = q['object_label']
            echo(f"  * {l}")
    
# Remove previous matches from label, with the goal of speeding up matching
# For example, if a PATO quality is matched, we removed it from the label 
def reduce_label(label, matches):
    label_reduced = label
    for q in matches:
        ### exceptions to this: we dont want to remove "cell" or similar ever from the
        # Label as this may cause issues, like not matching a cell type when you have
        # bipolar cell morphology and cell morphology as the quality
        q_label = q['match_string']
        q_label = q_label.replace("cell","")     
        label_reduced = label_reduced.replace(q_label,"")
    label_reduced = label_reduced.strip()
    return label_reduced

def to_sssom_df(list_of_mappings, name):
    prefix_map={
        'OBA': 'http://purl.obolibrary.org/obo/OBA_',
        'GO': 'http://purl.obolibrary.org/obo/GO_',
        'UBERON': 'http://purl.obolibrary.org/obo/UBERON_',
        'CL': 'http://purl.obolibrary.org/obo/CL_',
        'VT': 'http://purl.obolibrary.org/obo/VT_',
        'PATO': 'http://purl.obolibrary.org/obo/PATO_',
        'EFO': 'http://www.ebi.ac.uk/efo/EFO_',
        'oboInOwl': 'http://www.geneontology.org/formats/oboInOwl#',
        'rdfs': 'http://www.w3.org/2000/01/rdf-schema#', 
    }
    
    ## Create empty mapping set
    mapping_set_id = f"https://w3id.org/sssom/commons/phenotype/{name}"
    license = "https://creativecommons.org/licenses/by/4.0/"
    ms = MappingSet(mapping_set_id=mapping_set_id, license=license)

    ## Add mappings to Mapping Set
    ms.mappings = list_of_mappings
    
    ## Create Mapping set document
    doc = MappingSetDocument(mapping_set=ms, prefix_map=prefix_map)
    msdf = to_mapping_set_dataframe(doc)
    return msdf.df

def align_ontology(df_input, pato_labels=[], cl_labels=[], uberon_labels=[], go_labels=[], chebi_labels=[], head_count=0, alignment_id="unspecified"):
    # Preprocess input
    df = df_input.copy()

    if head_count > 0:
        df = df.head(head_count)
    df = df.reset_index()  # make sure indexes pair with number of rows
        
    # Declare output variables
    mlist: List[Mapping] = []

    dosdp_matches = []

    for index, row in df.iterrows():
        curie = row['id']
        mdict = {}
        
        cell_types = []
        bp_mf_ccs = []
        anatomical_entities = []
        chebi_entities = []
        
        
        predicate = row['predicate']
        label = prepare_label(row['value'])
        
        mdict['subject_id'] = curie
        mdict['subject_match_field'] = predicate
        mdict['subject_label'] = label
        mdict['match_type'] = 'Lexical'

        echo("")
        echo("--------------------------------")
        echo(f"{curie}: {label}")

        echo("")
        echo("Matching OBA")
        traits_oba = get_matches(label, oba_labels, strict_word_order=True, exact_only=True) # important!!

        if traits_oba:
            echo("")
            echo("Outcomes")
            for matches in [traits_oba]:
                for matches_mdict in matches:
                    echo(f"    {matches_mdict['object_id']} ({json.dumps(matches_mdict, sort_keys=True, indent=4)})")
                    matches_mdict.update(mdict)
                    mlist.append(Mapping(**matches_mdict))
        
        echo("")
        echo("Matching PATO")
        qualities = get_matches(label, pato_labels, exact_only=False)
        print_matches(qualities)

        if not qualities:
            echo(f"No quality matches for {label}, skipping rest..")
            continue

        # Make Label wo quality reference
        skip_next = False

        label_reduced = reduce_label(label, qualities)

        if len(label_reduced) < 3:
            echo(f"Label ({label_reduced}) now (after PATO) has less than 3 characters, skipping rest.")
            skip_next = True

        if not skip_next:
            echo("")
            echo("Matching CL")
            cell_types = get_matches(label_reduced, cl_labels, strict_word_order=True, exact_only=False)
            print_matches(cell_types)

            label_reduced = reduce_label(label_reduced, cell_types)

            if len(label_reduced) < 3:
                echo(f"Label ({label_reduced}) now (after CL) has less than 3 characters, skipping rest.")
                skip_next = True

        if not skip_next:
            echo("")
            echo("Matching Uberon")
            anatomical_entities = get_matches(label_reduced, uberon_labels, strict_word_order=True, exact_only=False)
            print_matches(anatomical_entities)

            label_reduced = reduce_label(label_reduced, anatomical_entities)

            if len(label_reduced) < 3:
                echo(f"Label ({label_reduced}) now (after Uberon) has less than 3 characters, skipping GO.")
                skip_next = True

        if not skip_next:
            echo("")
            echo("Matching CHEBI")
            chebi_entities = get_matches(label_reduced, chebi_labels, strict_word_order=True, exact_only=False)
            print_matches(chebi_entities)

            label_reduced = reduce_label(label_reduced, chebi_entities)

            if len(label_reduced) < 3:
                echo(f"Label ({label_reduced}) now (after CHEBI) has less than 3 characters, skipping GO.")
                skip_next = True

        if not skip_next:
            echo("")
            echo("Matching GO")
            bp_mf_ccs = get_matches(label_reduced, go_labels, exact_only=False)
            print_matches(bp_mf_ccs)

        unique_qualities = [qmatches['object_label'] for qmatches in qualities]
        unique_qualities = list(set(unique_qualities))
        for qmatches in qualities:
            pattern_dict = mdict.copy()
            pattern_dict['quality'] = qmatches['object_id']
            pattern_dict['quality_label'] = qmatches['object_label']
            if qmatches['object_label'] == "quality" and len(unique_qualities) > 1:
                echo("There is a match to PATO:quality, but other qualities exist.. Skipping this one.")
                continue
            for matches_list in [anatomical_entities, cell_types, bp_mf_ccs]:
                largest_e = []
                cache_es = {}
                for ematches in matches_list:
                    process = True
                    if len(largest_e) > 0:
                        for e in largest_e:
                            if e in ematches['object_label']:
                                # Bigger one found
                                largest_e.remove(e)
                                process = True
                                break
                            elif ematches['object_label'] in e:
                                process = False
                                break

                    if process:
                        largest_e.append(ematches['object_label'])
                        pattern_dict['attribute'] = ematches['object_id']
                        pattern_dict['attribute_label'] = ematches['object_label']
                        np = pattern_dict.copy()
                        cache_es[ematches['object_label']] = np
                for e in largest_e:
                    dosdp_matches.append(cache_es[e])
    
    df_mappings = pd.DataFrame(dosdp_matches)
    df_sssom = to_sssom_df(mlist, alignment_id)
    
    if len(df_mappings)>0:
        post_mapping_unmapped = df[~df['id'].isin(df_mappings['subject_id'])]
    else:
        post_mapping_unmapped = df

    return df_mappings, df_sssom, post_mapping_unmapped


## EFO - OBA alignment

In the following, we will attempt an EFO alignment with OBA. At the same time, we will try to patternise EFO classes, with the goal of including these newly patternised EFO classes straight into OBA and mapping them back to EFO.

The process goes like this:

For all measurement terms T in EFO, 

1. Try to match T to OBA (for the purpose of having a straight mapping)
3. Try to match a PATO term.
4. If 3 was successful, we also try to match an UBERON, CL and/or GO term.
5. EFO-OBA mappings are exported as SSSOM
6. EFO to EQ mappings are exported as a DOSDP pattern file with a bit of metadata

### Methods for the process

The central ideas in the mapping process are:

1. We remove noisy words like "trait" or "measurement" from our labels prior to alignment (we also do the usual preprocessing like lower casing etc)
2. For EFO-OBA/VT we take a naive process which attempts to only match _exactly_ the preprocessed labels
3. For the EFO-EQ patternisation we search for occurrences of E's in the label of the EFO term


### Prepare unmapped EFO elements

In [16]:
# We only care about unmapped elements!
# We look at all the efo classes that are neither mentioned in the oba to efo mappings we already have
# nor in the file with the exclusions

efo_unmapped = efo_labels[~efo_labels['id'].isin(oba_efo_sssom.df['subject_id'])].copy()
if len(efo_mapping_exclusions)>0:
    efo_unmapped = efo_unmapped[~efo_unmapped['id'].isin(efo_mapping_exclusions['subject_id'])].copy()
efo_unmapped

,id,predicate,value
0,EFO:0010138,rdfs:label,nitrite measurement
1,EFO:0020165,rdfs:label,AT-rich interactive domain-containing protein ...
2,EFO:0008214,rdfs:label,lymphotactin measurement
3,EFO:0005273,rdfs:label,sleep depth
4,EFO:0010773,rdfs:label,CD5 measurement
...,...,...,...
4966,EFO:0005117,oboInOwl:hasExactSynonym,"Regulated on Activation, Normal T cell Express..."
4967,EFO:0009272,oboInOwl:hasExactSynonym,VCA-IgG seropositivity
4968,EFO:0009272,oboInOwl:hasExactSynonym,VCA seropositivity
4969,EFO:0010381,oboInOwl:hasExactSynonym,PC 36:3


### Mapping

In [17]:
efo_dosdp_candidates, efo_mapping_candidates, oba_efo_mapping_unmapped = align_ontology(df_input=efo_unmapped, pato_labels=pato_labels, cl_labels=cl_labels, uberon_labels=uberon_labels, go_labels=go_labels, chebi_labels=chebi_labels, head_count=head_count)

# export the entirely unmapped data for future reference

save_tsv(efo_dosdp_candidates, oba_efo_dosdp_candidates_file)
save_tsv(efo_mapping_candidates, oba_efo_mapping_candidates_file)
save_tsv(oba_efo_mapping_unmapped, oba_efo_mapping_unmapped_file)


--------------------------------
EFO:0010138: nitrite quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL

Matching Uberon

Matching CHEBI
Matches:
  * nitrite
  * nitrite
Label () now (after CHEBI) has less than 3 characters, skipping GO.

--------------------------------
EFO:0020165: at-rich interactive domain-containing protein 3a quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL

Matching Uberon

Matching CHEBI
Matches:
  * protein

Matching GO

--------------------------------
EFO:0008214: lymphotactin quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL

Matching Uberon

Matching CHEBI

Matching GO

--------------------------------
EFO:0005273: sleep depth

Matching OBA

Matching PATO
Matches:
  * depth

Matching CL

Matching Uberon

Matching CHEBI

Matching GO
Matches:
  * sleep

--------------------------------
EFO:0010773: cd5 quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL

Matching Uberon

Matc


Matching PATO
Matches:
  * quality

Matching CL

Matching Uberon

Matching CHEBI
Matches:
  * protein

Matching GO

--------------------------------
EFO:0004467: insulin quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL

Matching Uberon

Matching CHEBI
Matches:
  * insulin
Label () now (after CHEBI) has less than 3 characters, skipping GO.

--------------------------------
EFO:0004767: visceral:subcutaneous adipose tissue ratio

Matching OBA

Matching PATO
Matches:
  * ratio
  * subcutaneous

Matching CL

Matching Uberon
Matches:
  * adipose tissue
  * tissue

Matching CHEBI

Matching GO

--------------------------------
EFO:0021370: x-14473 quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL

Matching Uberon

Matching CHEBI

Matching GO

--------------------------------
EFO:0008069: carboxypeptidase b2 quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL

Matching Uberon

Matching CHEBI

Matching GO

--------------------------


Matching PATO
Matches:
  * quality
  * weight

Matching CL

Matching Uberon

Matching CHEBI

Matching GO
There is a match to PATO:quality, but other qualities exist.. Skipping this one.

--------------------------------
EFO:0021419: myristate 14:0-to-myristoleate 14:1n5 ratio

Matching OBA

Matching PATO
Matches:
  * ratio

Matching CL

Matching Uberon

Matching CHEBI

Matching GO

--------------------------------
EFO:0008093: complement component c7 quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL

Matching Uberon

Matching CHEBI

Matching GO

--------------------------------
EFO:0004328: exercise test

Matching OBA

Matching PATO
No quality matches for exercise test, skipping rest..

--------------------------------
EFO:0020701: receptor tyrosine-protein kinase erbb-3 quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL

Matching Uberon

Matching CHEBI

Matching GO

--------------------------------
EFO:0020395: fractalkine quality

Matching OB

Matches:
  * pancreas
Label () now (after Uberon) has less than 3 characters, skipping GO.

--------------------------------
EFO:0020237: casein kinase ii 2-alpha':2-beta heterotetramer quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL

Matching Uberon

Matching CHEBI

Matching GO

--------------------------------
EFO:0007856: facial height quality

Matching OBA

Matching PATO
Matches:
  * quality
  * height

Matching CL

Matching Uberon

Matching CHEBI

Matching GO
There is a match to PATO:quality, but other qualities exist.. Skipping this one.

--------------------------------
EFO:0007881: job-related exhaustion quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL

Matching Uberon

Matching CHEBI

Matching GO

--------------------------------
EFO:0021334: x-12786 quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL

Matching Uberon

Matching CHEBI

Matching GO

--------------------------------
EFO:0021405: x-11444-to-x-12844 ra


Matching CHEBI
Matches:
  * phenylalanine
  * Phenylalanine
  * phenylalanine

Matching GO

--------------------------------
EFO:0009097: positive domain quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL

Matching Uberon

Matching CHEBI

Matching GO

--------------------------------
EFO:0020152: anterior gradient protein 2 homolog quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL

Matching Uberon

Matching CHEBI
Matches:
  * protein

Matching GO

--------------------------------
EFO:0007997: granulocyte percentage of myeloid white cells

Matching OBA

Matching PATO
Matches:
  * white

Matching CL
Matches:
  * granulocyte

Matching Uberon

Matching CHEBI

Matching GO

--------------------------------
EFO:0020484: interferon gamma receptor 1 quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL

Matching Uberon

Matching CHEBI
Matches:
  * interferon

Matching GO

--------------------------------
EFO:0021320: x-12680 quality

Ma

...


Matching PATO
Matches:
  * quality

Matching CL

Matching Uberon

Matching CHEBI

Matching GO

--------------------------------
EFO:0004618: phylloquinone quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL

Matching Uberon

Matching CHEBI
Matches:
  * phylloquinone
  * phylloquinone
Label () now (after CHEBI) has less than 3 characters, skipping GO.

--------------------------------
EFO:0009237: platelets

Matching OBA

Matching PATO
No quality matches for platelets, skipping rest..

--------------------------------
EFO:0009237: estimated

Matching OBA

Matching PATO
No quality matches for estimated, skipping rest..

--------------------------------
EFO:0009237: platest

Matching OBA

Matching PATO
No quality matches for platest, skipping rest..

--------------------------------
EFO:0600042: serum clozapine-to-norclozapine ratio quality

Matching OBA

Matching PATO
Matches:
  * quality
  * ratio

Matching CL

Matching Uberon
Matches:
  * serum

Matching CHEBI

Matc


--------------------------------
EFO:0008056: interferon gamma-induced protein 10 quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL

Matching Uberon

Matching CHEBI
Matches:
  * protein
  * interferon

Matching GO

--------------------------------
EFO:0008056: ip10 quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL

Matching Uberon

Matching CHEBI

Matching GO

--------------------------------
EFO:0008056: ip-10 quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL

Matching Uberon

Matching CHEBI

Matching GO

--------------------------------
EFO:0010385: pc 38:3

Matching OBA

Matching PATO
No quality matches for pc 38:3, skipping rest..

--------------------------------
EFO:0010399: tag 44:1

Matching OBA

Matching PATO
No quality matches for tag 44:1, skipping rest..

--------------------------------
EFO:0008592: hdl particle size quality

Matching OBA

Matching PATO
Matches:
  * quality
  * size

Matching CL

Matching Uber


Matching CHEBI

Matching GO

--------------------------------
EFO:0010057: svcam-1 quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL

Matching Uberon

Matching CHEBI

Matching GO

--------------------------------
EFO:0010057: soluble vascular cell adhesion molecule-1 quality

Matching OBA

Matching PATO
Matches:
  * quality
  * soluble

Matching CL
Matches:
  * cell

Matching Uberon

Matching CHEBI

Matching GO
Matches:
  * cell adhesion
There is a match to PATO:quality, but other qualities exist.. Skipping this one.

--------------------------------
EFO:0010057: vcam-1 quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL

Matching Uberon

Matching CHEBI

Matching GO

--------------------------------
EFO:0005192: rdw

Matching OBA

Matching PATO
No quality matches for rdw, skipping rest..

--------------------------------
EFO:0009228: ret. corpuscular hgb concentration mean

Matching OBA

Matching PATO
Matches:
  * concentration

Matching CL

Mat


Matching PATO
No quality matches for aphasia tests, skipping rest..

--------------------------------
EFO:0003926: test, neuropsychologic

Matching OBA

Matching PATO
No quality matches for test, neuropsychologic, skipping rest..

--------------------------------
EFO:0003926: memory for designs test

Matching OBA

Matching PATO
No quality matches for memory for designs test, skipping rest..

--------------------------------
EFO:0009358: hbs igg quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL

Matching Uberon

Matching CHEBI

Matching GO

--------------------------------
EFO:0600092: fhl-1 protein quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL

Matching Uberon

Matching CHEBI
Matches:
  * protein

Matching GO

--------------------------------
EFO:0004622: sphingolipid assay

Matching OBA

Matching PATO
No quality matches for sphingolipid assay, skipping rest..

--------------------------------
EFO:0010427: tag 54:9

Matching OBA

Matching 


Matching PATO
No quality matches for hs-ctnt levels, skipping rest..

--------------------------------
EFO:0005936: underweight bmi status

Matching OBA

Matching PATO
No quality matches for underweight bmi status, skipping rest..

--------------------------------
EFO:0008401: susceptibility to herpes zoster quality

Matching OBA

Matching PATO
Matches:
  * quality
  * susceptibility

Matching CL

Matching Uberon

Matching CHEBI

Matching GO
There is a match to PATO:quality, but other qualities exist.. Skipping this one.

--------------------------------
EFO:0011031: gut permeability quality

Matching OBA

Matching PATO
Matches:
  * permeability
  * quality

Matching CL

Matching Uberon

Matching CHEBI

Matching GO
There is a match to PATO:quality, but other qualities exist.. Skipping this one.

--------------------------------
EFO:0007001: dhea-s quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL

Matching Uberon

Matching CHEBI

Matching GO

---------------------

### Results

In [18]:
display(HTML("<h4>DOSDP mapping candidates</h4>"))
display(HTML(efo_dosdp_candidates.to_html()))

display(HTML("<h4>SSSOM mapping candidates</h4>"))
display(HTML(efo_mapping_candidates.to_html()))

display(HTML("<h4>Top 10 unmapped candidates</h4>"))
display(HTML(oba_efo_mapping_unmapped.head(10).to_html()))

,subject_id,subject_match_field,subject_label,match_type,quality,quality_label,attribute,attribute_label
0,EFO:0005273,rdfs:label,sleep depth,Lexical,PATO:0001595,depth,GO:0030431,sleep
1,EFO:0007719,rdfs:label,carotid artery external diameter quality,Lexical,PATO:0001334,diameter,UBERON:0005396,carotid artery
2,EFO:0009230,rdfs:label,reticulocyte corpuscular hemoglobin distribution width,Lexical,PATO:0000921,width,CL:0000558,reticulocyte
3,EFO:0009230,rdfs:label,reticulocyte corpuscular hemoglobin distribution width,Lexical,PATO:0000060,distribution,CL:0000558,reticulocyte
4,EFO:0020760,rdfs:label,t-cell surface glycoprotein cd4 quality,Lexical,PATO:0000001,quality,CL:0000084,T-cell
5,EFO:0020760,rdfs:label,t-cell surface glycoprotein cd4 quality,Lexical,PATO:0000001,quality,GO:0009986,cell surface
6,EFO:0020354,rdfs:label,ephrin type-b receptor 4 quality,Lexical,PATO:0000001,quality,GO:0005106,ephrin
7,EFO:0007579,rdfs:label,blood chromium quality,Lexical,PATO:0000001,quality,UBERON:0000178,blood
8,EFO:0009238,rdfs:label,immature plasma cell count,Lexical,PATO:0001501,immature,CL:0000786,plasma cell
9,EFO:0004767,rdfs:label,visceral:subcutaneous adipose tissue ratio,Lexical,PATO:0001470,ratio,UBERON:0001013,adipose tissue


""


,index,id,predicate,value
0,0,EFO:0010138,rdfs:label,nitrite measurement
1,1,EFO:0020165,rdfs:label,AT-rich interactive domain-containing protein 3A measurement
2,2,EFO:0008214,rdfs:label,lymphotactin measurement
4,4,EFO:0010773,rdfs:label,CD5 measurement
5,5,EFO:0010067,rdfs:label,corneal resistance factor
6,6,EFO:0021281,rdfs:label,X-12040 measurement
7,7,EFO:0010516,rdfs:label,orotic acid measurement@en
8,8,EFO:0020716,rdfs:label,scavenger receptor class F member 2 measurement
11,11,EFO:0021024,rdfs:label,levulinate 4-oxovalerate measurement
12,13,EFO:0010626,rdfs:label,placenta growth factor measurement@en


#### Distribution of PATO qualities matched

This can help identifying the next set of entities to work on.

In [19]:
if len(efo_dosdp_candidates)>0:
    df_quals = efo_dosdp_candidates['quality_label'].value_counts().rename_axis('unique_values').reset_index(name='counts')
    echo(df_quals.head(20))

    unique_values  counts
0         quality     520
1          volume      90
2           ratio      39
3           white      20
4           width      14
5    distribution      14
6       thickness      11
7        pressure      11
8            rate       9
9            size       8
10  concentration       7
11     morphology       7
12           area       6
13        content       6
14        soluble       5
15       immature       5
16      frequency       5
17           left       4
18          fatty       4
19        dentate       4


#### Matches involving "Volume"

See https://github.com/obophenotype/bio-attribute-ontology/issues/97

In [20]:
if len(efo_dosdp_candidates)>0:
    df_vol=efo_dosdp_candidates[efo_dosdp_candidates['quality_label']=='volume']
    display(HTML(df_vol.to_html()))

,subject_id,subject_match_field,subject_label,match_type,quality,quality_label,attribute,attribute_label
13,EFO:0010326,rdfs:label,precuneus cortex volume quality@en,Lexical,PATO:0000918,volume,UBERON:0006093,precuneus cortex
16,EFO:0009405,rdfs:label,parasubiculum volume,Lexical,PATO:0000918,volume,UBERON:0004683,parasubiculum
20,EFO:0600045,rdfs:label,pancreas volume,Lexical,PATO:0000918,volume,UBERON:0001264,pancreas
25,EFO:0010330,rdfs:label,superior parietal cortex volume quality@en,Lexical,PATO:0000918,volume,UBERON:0006094,superior parietal cortex
47,EFO:0010317,rdfs:label,paracentral lobule volume quality@en,Lexical,PATO:0000918,volume,UBERON:0035933,paracentral lobule
54,EFO:0006930,rdfs:label,brain volume quality,Lexical,PATO:0000918,volume,UBERON:0000955,brain
57,EFO:0010335,rdfs:label,third ventricle volume quality@en,Lexical,PATO:0000918,volume,UBERON:0002286,third ventricle
70,EFO:0010604,rdfs:label,urine volume quality@en,Lexical,PATO:0000918,volume,UBERON:0001088,urine
80,EFO:0010304,rdfs:label,frontal pole volume quality@en,Lexical,PATO:0000918,volume,UBERON:0002795,frontal pole
81,EFO:0010147,rdfs:label,temporal lobe volume quality,Lexical,PATO:0000918,volume,UBERON:0001871,temporal lobe


## VT - OBA alignment

### Prepare unmapped VT terms

In [22]:

# Check if some have been missed (IRI analysis)
vt_oba_mapping = oba_labels.copy()
vt_oba_mapping['subject_id'] = vt_oba_mapping[vt_oba_mapping['id'].str.contains(":VT")]['id'].str.replace("OBA:VT","VT:")
echo(len(vt_oba_mapping))

# Identify those that have an VT mappings based on the OBO IRI
vt_oba_mapping = vt_oba_mapping[~vt_oba_mapping['subject_id'].isna()]
echo(len(vt_oba_mapping))

# Get the set of mappings that are not yet in the official OBA-VT sssom mapping file
vt_id_mapped = vt_oba_mapping[~vt_oba_mapping['subject_id'].isin(oba_vt_sssom.df['subject_id'])].copy()

# exclude all VT terms already fully mapped in the sssom file
vt_unmapped = vt_labels[~vt_labels['id'].isin(oba_vt_sssom.df['subject_id'])].copy()
echo(len(vt_unmapped))

# exclude all VT terms that are trivially mapped through ID mapping
vt_unmapped = vt_unmapped[~vt_unmapped['id'].isin(vt_oba_mapping['subject_id'])].copy()
echo(len(vt_unmapped))

### Removed excluded terms from unmapped VT set
if len(vt_mapping_exclusions)>0:
    vt_unmapped = vt_unmapped[~vt_unmapped.id.isin(vt_mapping_exclusions['subject_id'])].copy()

# export already mapped VT ids that are not in sssom yet as SSSOM
vt_id_mapped['predicate_id']="skos:exactMatch"
vt_id_mapped['object_id']=vt_id_mapped['id']
vt_id_mapped['match_type']="HumanCurated"
vt_id_mapped = vt_id_mapped[['subject_id','predicate_id','object_id','match_type']].copy().drop_duplicates()
save_tsv(vt_id_mapped, oba_vt_mapping_missed_file)


15399
4560
2080
2080


### Mapping

In [23]:
vt_dosdp_candidates, vt_mapping_candidates, vt_efo_mapping_unmapped = align_ontology(df_input=vt_unmapped, pato_labels=pato_labels, cl_labels=cl_labels, uberon_labels=uberon_labels, go_labels=go_labels, chebi_labels=chebi_labels, head_count=head_count)

# export the entirely unmapped data for future reference

save_tsv(vt_dosdp_candidates, oba_vt_dosdp_candidates_file)
save_tsv(vt_mapping_candidates, oba_vt_mapping_candidates_file)
save_tsv(vt_efo_mapping_unmapped, oba_vt_mapping_unmapped_file)



--------------------------------
VT:0003477: nerve fiber response quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL

Matching Uberon
Matches:
  * nerve
  * nerve fiber

Matching CHEBI

Matching GO

--------------------------------
VT:0003799: macrophage migration quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL
Matches:
  * macrophage

Matching Uberon

Matching CHEBI

Matching GO

--------------------------------
VT:0010765: head and neck integrity quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL

Matching Uberon
Matches:
  * neck
  * head

Matching CHEBI

Matching GO

--------------------------------
VT:2000004: coronary blood flow quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL

Matching Uberon
Matches:
  * blood

Matching CHEBI

Matching GO

--------------------------------
VT:0000066: pancreas/salivary gland physiology quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL

Matc


Matching GO
Matches:
  * memory
There is a match to PATO:quality, but other qualities exist.. Skipping this one.

--------------------------------
VT:0003424: neuronal precursor differentiation quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL

Matching Uberon

Matching CHEBI

Matching GO

--------------------------------
VT:0010053: muscle fatty acid c18:0 amount

Matching OBA

Matching PATO
Matches:
  * amount
  * fatty

Matching CL

Matching Uberon

Matching CHEBI
Matches:
  * acid
  * Acid
  * acid

Matching GO

--------------------------------
VT:0008112: monocyte development quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL
Matches:
  * monocyte

Matching Uberon

Matching CHEBI

Matching GO

--------------------------------
VT:0010163: interleukin-2 physiology quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL

Matching Uberon

Matching CHEBI

Matching GO

--------------------------------
VT:0003289: intestinal perist


Matching PATO
Matches:
  * amount
  * fatty

Matching CL

Matching Uberon
Matches:
  * milk

Matching CHEBI
Matches:
  * acid
  * Acid
  * acid

Matching GO

--------------------------------
VT:0002272: nervous system electrophysiology quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL

Matching Uberon
Matches:
  * nervous system

Matching CHEBI

Matching GO

--------------------------------
VT:0001213: skin cell quantity

Matching OBA

Matching PATO
No quality matches for skin cell quantity, skipping rest..

--------------------------------
VT:0008497: blood immunoglobulin g2b amount

Matching OBA

Matching PATO
Matches:
  * amount

Matching CL

Matching Uberon
Matches:
  * blood

Matching CHEBI

Matching GO

--------------------------------
VT:0000944: cochlear outer hair cell afferent innervation morphology quality

Matching OBA

Matching PATO
Matches:
  * quality
  * morphology

Matching CL
Matches:
  * cell
  * cochlear outer hair cell

Matching Uberon

Matchi


Matching PATO
Matches:
  * amount

Matching CL

Matching Uberon
Matches:
  * medulla oblongata
  * medulla

Matching CHEBI
Matches:
  * serotonin
Label () now (after CHEBI) has less than 3 characters, skipping GO.

--------------------------------
VT:0010011: femur mineral mass

Matching OBA

Matching PATO
Matches:
  * mass

Matching CL

Matching Uberon
Matches:
  * femur

Matching CHEBI
Matches:
  * mineral
  * mineral
Label () now (after CHEBI) has less than 3 characters, skipping GO.

--------------------------------
VT:0010904: tissue oxygen amount

Matching OBA

Matching PATO
Matches:
  * amount

Matching CL

Matching Uberon
Matches:
  * tissue

Matching CHEBI
Matches:
  * oxygen
Label () now (after CHEBI) has less than 3 characters, skipping GO.

--------------------------------
VT:0006227: iris development quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL

Matching Uberon
Matches:
  * iris

Matching CHEBI

Matching GO

--------------------------------
VT:00


Matching PATO
Matches:
  * amount

Matching CL

Matching Uberon
Matches:
  * spinal cord
  * cervical spinal cord
  * cervical spinal cord

Matching CHEBI
Matches:
  * acid
  * Acid
  * acid

Matching GO

--------------------------------
VT:0005228: vertebral body/neural arch fusion quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL

Matching Uberon

Matching CHEBI

Matching GO

--------------------------------
VT:0001450: vestibular hair bundle inter-stereocilial link morphology quality

Matching OBA

Matching PATO
Matches:
  * quality
  * morphology

Matching CL

Matching Uberon
Matches:
  * hair

Matching CHEBI

Matching GO
There is a match to PATO:quality, but other qualities exist.. Skipping this one.

--------------------------------
VT:0003744: orofacial morphology quality

Matching OBA

Matching PATO
Matches:
  * quality
  * morphology

Matching CL

Matching Uberon

Matching CHEBI

Matching GO
There is a match to PATO:quality, but other qualities exist.. Sk

...


Matching PATO
Matches:
  * quality

Matching CL

Matching Uberon

Matching CHEBI

Matching GO

--------------------------------
VT:0008668: interleukin-23b secretion

Matching OBA

Matching PATO
No quality matches for interleukin-23b secretion, skipping rest..

--------------------------------
VT:0008668: il-23p40 secretion

Matching OBA

Matching PATO
No quality matches for il-23p40 secretion, skipping rest..

--------------------------------
VT:0008668: interleukin-12 beta chain secretion

Matching OBA

Matching PATO
No quality matches for interleukin-12 beta chain secretion, skipping rest..

--------------------------------
VT:0008668: interleukin-12p40 secretion

Matching OBA

Matching PATO
No quality matches for interleukin-12p40 secretion, skipping rest..

--------------------------------
VT:0008668: il-12b secretion

Matching OBA

Matching PATO
No quality matches for il-12b secretion, skipping rest..

--------------------------------
VT:0008668: il-23b secretion

Matching OBA



Matches:
  * brain

Matching CHEBI
Matches:
  * sterol

Matching GO

--------------------------------
VT:0000979: hair down neuron morphology quality

Matching OBA

Matching PATO
Matches:
  * down
  * quality
  * morphology

Matching CL
Matches:
  * neuron

Matching Uberon
Matches:
  * hair
Label () now (after Uberon) has less than 3 characters, skipping GO.
There is a match to PATO:quality, but other qualities exist.. Skipping this one.

--------------------------------
VT:0000979: d-hair receptor morphology quality

Matching OBA

Matching PATO
Matches:
  * quality
  * morphology

Matching CL

Matching Uberon

Matching CHEBI

Matching GO
There is a match to PATO:quality, but other qualities exist.. Skipping this one.

--------------------------------
VT:1000519: adipose eicosadienoic acid concentration

Matching OBA

Matching PATO
Matches:
  * concentration

Matching CL

Matching Uberon

Matching CHEBI
Matches:
  * acid
  * Acid
  * acid

Matching GO

--------------------------------



Matching PATO
Matches:
  * concentration

Matching CL

Matching Uberon

Matching CHEBI
Matches:
  * acid
  * Acid
  * acid

Matching GO

--------------------------------
VT:0001173: bulbuorethral gland ductal cell number

Matching OBA

Matching PATO
No quality matches for bulbuorethral gland ductal cell number, skipping rest..

--------------------------------
VT:0001173: cowper's gland ductal cell number

Matching OBA

Matching PATO
No quality matches for cowper's gland ductal cell number, skipping rest..

--------------------------------
VT:0010376: milk soluble calcium content

Matching OBA

Matching PATO
Matches:
  * soluble
  * content

Matching CL

Matching Uberon
Matches:
  * milk

Matching CHEBI
Matches:
  * calcium
  * calcium
Label () now (after CHEBI) has less than 3 characters, skipping GO.

--------------------------------
VT:0008073: cd4-positive t cell number

Matching OBA

Matching PATO
No quality matches for cd4-positive t cell number, skipping rest..

---------------


Matching GO

--------------------------------
VT:0010210: depth of the glandular crypts of the duodenum

Matching OBA

Matching PATO
Matches:
  * glandular
  * depth

Matching CL

Matching Uberon
Matches:
  * duodenum

Matching CHEBI

Matching GO

--------------------------------
VT:0003876: cerebral physiology quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL

Matching Uberon

Matching CHEBI

Matching GO

--------------------------------
VT:0003248: glutaminergic neuron number

Matching OBA

Matching PATO
No quality matches for glutaminergic neuron number, skipping rest..

--------------------------------
VT:0003248: glutamate neuron number

Matching OBA

Matching PATO
No quality matches for glutamate neuron number, skipping rest..

--------------------------------
VT:0010138: lymphatic vessel cell number

Matching OBA

Matching PATO
No quality matches for lymphatic vessel cell number, skipping rest..

--------------------------------
VT:0008038: nkt cell number



Matching CHEBI

Matching GO
Matches:
  * cell physiology

--------------------------------
VT:0008042: nk t lymphocyte physiology quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL
Matches:
  * lymphocyte
  * T lymphocyte

Matching Uberon

Matching CHEBI

Matching GO

--------------------------------
VT:0008042: nkt cell physiology quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL
Matches:
  * cell

Matching Uberon

Matching CHEBI

Matching GO
Matches:
  * cell physiology

--------------------------------
VT:0008042: natural killer t lymphocyte physiology quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL
Matches:
  * lymphocyte
  * T lymphocyte

Matching Uberon

Matching CHEBI

Matching GO

--------------------------------
VT:0008042: natural killer t-cell physiology quality

Matching OBA

Matching PATO
Matches:
  * quality

Matching CL
Matches:
  * T-cell

Matching Uberon

Matching CHEBI

Matching GO
Matches:
  * cell phys

### Results

In [24]:
display(HTML("<h4>DOSDP mapping candidates</h4>"))
display(HTML(efo_dosdp_candidates.to_html()))

display(HTML("<h4>SSSOM mapping candidates</h4>"))
display(HTML(efo_mapping_candidates.to_html()))

display(HTML("<h4>Top 10 unmapped candidates</h4>"))
display(HTML(vt_efo_mapping_unmapped.head(10).to_html()))

,subject_id,subject_match_field,subject_label,match_type,quality,quality_label,attribute,attribute_label
0,EFO:0005273,rdfs:label,sleep depth,Lexical,PATO:0001595,depth,GO:0030431,sleep
1,EFO:0007719,rdfs:label,carotid artery external diameter quality,Lexical,PATO:0001334,diameter,UBERON:0005396,carotid artery
2,EFO:0009230,rdfs:label,reticulocyte corpuscular hemoglobin distribution width,Lexical,PATO:0000921,width,CL:0000558,reticulocyte
3,EFO:0009230,rdfs:label,reticulocyte corpuscular hemoglobin distribution width,Lexical,PATO:0000060,distribution,CL:0000558,reticulocyte
4,EFO:0020760,rdfs:label,t-cell surface glycoprotein cd4 quality,Lexical,PATO:0000001,quality,CL:0000084,T-cell
5,EFO:0020760,rdfs:label,t-cell surface glycoprotein cd4 quality,Lexical,PATO:0000001,quality,GO:0009986,cell surface
6,EFO:0020354,rdfs:label,ephrin type-b receptor 4 quality,Lexical,PATO:0000001,quality,GO:0005106,ephrin
7,EFO:0007579,rdfs:label,blood chromium quality,Lexical,PATO:0000001,quality,UBERON:0000178,blood
8,EFO:0009238,rdfs:label,immature plasma cell count,Lexical,PATO:0001501,immature,CL:0000786,plasma cell
9,EFO:0004767,rdfs:label,visceral:subcutaneous adipose tissue ratio,Lexical,PATO:0001470,ratio,UBERON:0001013,adipose tissue


""


,index,id,predicate,value
6,14,VT:0006230,rdfs:label,iris stroma cell quantity
13,27,VT:0002327,rdfs:label,respiratory function trait
14,29,VT:0002911,rdfs:label,inhibitory postsynaptic potential
21,49,VT:0001964,rdfs:label,auditory threshold
22,50,VT:0010060,rdfs:label,muscle fatty acid cis-9-C16:1 amount
23,54,VT:0010696,rdfs:label,preovulatory follicle quantity
25,60,VT:0003148,rdfs:label,cochlear coil quantity
30,70,VT:1000720,rdfs:label,nonfunctional nipple quantity
34,78,VT:0003424,rdfs:label,neuronal precursor differentiation trait
35,79,VT:0010053,rdfs:label,muscle fatty acid C18:0 amount


In [ ]:
vt_efo_mapping_unmapped


For how to best deal with this complex curation worfklow, see OBA documentation.